<a href="https://colab.research.google.com/github/sileod/pragmeval/blob/master/run_pragmeval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.system('pip install transformers datasets pygrove xpflow funcy')
import pygrove
from xpflow import Xp
from datasets import get_dataset_config_names

In [ ]:
metric=load_metric("glue","mnli")

def preprocess_function(examples):
    if 'sentence1' in examples:
        return tokenizer(examples['sentence1'], examples['sentence2'], truncation=True, padding="max_length",max_length=256)
    else:
        return tokenizer(examples['sentence'], truncation=True, padding="max_length",max_length=256)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)
"""
Replace model_name by your model and adjust gradient_accumulation_steps to suit your gpu size
"""
class evaluate(Xp):
    model_name='roberta-base' # Your model
    dataset_name=get_dataset_config_names('pragmeval')
    num_train_epochs=3
    batch_size=16
    learning_rate=2e-5
    gradient_accumulation_steps=1
    warmup_ratio=0.06
    seed=0


for xp in evaluate():
    tokenizer=AutoTokenizer.from_pretrained(xp.model_name)
    dataset=load_dataset('pragmeval',xp.dataset_name)
    encoded_dataset = dataset.map(preprocess_function, batched=True)
    num_labels=dataset['train'].features['label'].num_classes
    model=AutoModelForSequenceClassification.from_pretrained(xp.model_name,num_labels=num_labels)

    args = TrainingArguments(
                xp.dataset_name,
                save_strategy="no",
                save_steps=1e9,
                evaluation_strategy = "epoch",
                per_device_train_batch_size=xp.batch_size//xp.gradient_accumulation_steps,
                per_device_eval_batch_size=xp.batch_size//xp.gradient_accumulation_steps,
                **fc.project(dict(xp), dir(TrainingArguments)),
                fp16_opt_level="O1",
            )
    trainer = Trainer(
        model,
        args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    trainer.train()
    print(xp, trainer.predict(encoded_datasets["test"]).metrics) # log your results here